# 1: Load relevant libraries

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 2: Load the data and make sure there are no extra index columns. Show the shape of the data AND the head of the data

In [64]:
data = pd.read_excel('messed_up_iris.xlsx',index_col=0)

# print the shape of the data
print('shape:',data.shape)

print('')

# print the head
print('head:')
print(data.head())

shape: (150, 7)

head:
   sepal_length  sepal_width  petal_length  petal_width species   color  origin
0           5.1          3.5           1.4          0.2  setosa   green     usa
1           4.9          3.0           1.4          0.2  setosa  yellow     usa
2           4.7          3.2           1.3          0.2  setosa   green     usa
3           4.6          3.1           1.5          0.2  setosa  orange   japan
4           5.0          3.6           1.4          0.2  setosa    blue  europe


# 3: Remove columns and rows where there are more than 50% of the data missing. Show the shape of the data after you have removed those columns/rows

In [65]:
# drop rows with more than 50% of the data missing
drop_nan = data.dropna(thresh = len(data.columns)/2, axis = 0)

# drop columns with more than 50% of the data missing
drop2_nan = drop_nan.dropna(thresh = len(data.index)/2, axis = 1)

# show the shape of the data
print('shape:',drop2_nan.shape)

shape: (145, 6)


# 4: Remove duplicate data, if there is any. Show the shape of the data

In [66]:
# check the duplicated rows
drop2_nan[drop2_nan.duplicated()]   # there are 5 duplicated rows

# drop the duplicated rows
nodupes = drop2_nan.drop_duplicates(ignore_index=True)

# show the shape of the data
print('shape:', nodupes.shape)

shape: (140, 6)


# 5: Dummy code the categorical data. Show the head of the data

In [73]:
# categorical data:species and origin

# fix typos in the data
nodupes['species'] = np.where(nodupes['species']=='virginia','virginica',nodupes['species'])
nodupes['species'] = np.where(nodupes['species']=='west virginia','virginica',nodupes['species'])
nodupes['species'] = np.where(nodupes['species']=='versicolr','versicolor',nodupes['species'])
nodupes['species'] = np.where(nodupes['species']=='seotsa','setosa',nodupes['species'])

nodupes['origin'] = np.where(nodupes['origin']=='uas','usa',nodupes['origin'])
nodupes['origin'] = np.where(nodupes['origin']=='euarope','europe',nodupes['origin'])

# dummy code the data
species_df = pd.get_dummies(nodupes['species'],drop_first=True)
origin_df = pd.get_dummies(nodupes['origin'],drop_first=True)

# concatenate into one dataframe
new_data = pd.concat([nodupes,species_df,origin_df], axis = 1)

# show the head of the data
new_data.head()

,sepal_length,sepal_width,petal_length,petal_width,species,origin,versicolor,virginica,japan,usa
0,5.1,3.5,1.4,0.2,setosa,usa,0,0,0,1
1,4.9,3.0,1.4,0.2,setosa,usa,0,0,0,1
2,4.7,3.2,1.3,0.2,setosa,usa,0,0,0,1
3,4.6,3.1,1.5,0.2,setosa,japan,0,0,1,0
4,5.0,3.6,1.4,0.2,setosa,europe,0,0,0,0


# 6: Drop the redundant columns from the dataframe so that you are left with just the dummy coded columns. Show the head of the data

In [68]:
# drop species and origins
new_data = new_data.drop(columns=['species','origin'])
new_data.head()

,sepal_length,sepal_width,petal_length,petal_width,versicolor,virginica,japan,usa
0,5.1,3.5,1.4,0.2,0,0,0,1
1,4.9,3.0,1.4,0.2,0,0,0,1
2,4.7,3.2,1.3,0.2,0,0,0,1
3,4.6,3.1,1.5,0.2,0,0,1,0
4,5.0,3.6,1.4,0.2,0,0,0,0


# 7: Remove outliers above 2 standard deviations.

In [69]:
# create a function that replaces outliers with NaN
def outliers(series):
  threshold_upper = series.mean() + (2 * series.std())  # 2 std above the mean
  threshold_lower = series.mean() - (2 * series.std())  # 2 std below the mean

  return np.where((series > threshold_upper) | (series < threshold_lower),np.nan,series)

# remove the outliers using the function
for column in new_data.columns:
  if isinstance(new_data[column][0],float)==1: 
    new_data[column] = outliers(new_data[column])

# 8: For the remaining missing data, replace with the median value.

In [70]:
fill_nans = new_data.fillna(new_data.median())

# 9: Bonus 0.5 points – Bin or bucket the petal width column into low, medium, and high depending on the following bins = 0 to 0.9, 0.9 to 1.8, 1.8 to 2.7. These should not be dummy coded, but instead, you should have one column with pedal_width as 1 for 0 to 0.9, 2 for 0.9 to 1.8, and 3 for 1.8 to 2.7.

In [71]:
petal_ordinal = []

final_data = fill_nans.copy()

# bin the data WITHOUT dummy coding
for i in final_data['petal_width']:
  if 0<i<=0.9:  # low
    petal_ordinal.append(1)
  if 0.9<i<=1.8:   # medium
    petal_ordinal.append(2)
  if 1.8<i<=2.7:   # high
    petal_ordinal.append(3)
final_data.insert(len(final_data.columns),'petal_ordinal',petal_ordinal)

# show the data
final_data.head(7)

,sepal_length,sepal_width,petal_length,petal_width,versicolor,virginica,japan,usa,petal_ordinal
0,5.1,3.5,1.4,0.2,0,0,0,1,1
1,4.9,3.0,1.4,0.2,0,0,0,1,1
2,4.7,3.2,1.3,0.2,0,0,0,1,1
3,4.6,3.1,1.5,0.2,0,0,1,0,1
4,5.0,3.6,1.4,0.2,0,0,0,0,1
5,5.4,3.9,1.7,0.4,0,0,0,0,1
6,4.6,3.4,1.4,1.3,0,0,0,0,2


# 10: Show (i.e., print) that there are no missing data and also use the describe method to show off the final dataset.

In [72]:
print('do any of the columns have missing data?')
print(final_data.isnull().any())

print('')
final_data.describe()

do any of the columns have missing data?
sepal_length     False
sepal_width      False
petal_length     False
petal_width      False
versicolor       False
virginica        False
japan            False
usa              False
petal_ordinal    False
dtype: bool



,sepal_length,sepal_width,petal_length,petal_width,versicolor,virginica,japan,usa,petal_ordinal
count,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000
mean,5.872143,3.057857,3.790000,1.209286,0.307143,0.350000,0.300000,0.364286,1.907143
std,0.791508,0.431505,1.718134,0.757590,0.462966,0.478682,0.459903,0.482957,0.738426
min,4.300000,2.000000,1.000000,0.100000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,5.200000,2.800000,1.575000,0.300000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,5.800000,3.000000,4.400000,1.300000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,6.400000,3.300000,5.100000,1.800000,1.000000,1.000000,1.000000,1.000000,2.000000
max,7.900000,4.400000,6.900000,2.500000,1.000000,1.000000,1.000000,1.000000,3.000000
